In [71]:
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os

In [44]:
df6 = pd.read_csv('../data/PSNI_StreetCrime_1Year/2017-04/2017-04-northern-ireland-street.csv')

In [45]:
df6

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context
0,NaN,2017-04,Police Service of Northern Ireland,Police Service of Northern Ireland,-6.051118,54.554806,On or near,NaN,NaN,Anti-social behaviour,NaN,NaN
1,NaN,2017-04,Police Service of Northern Ireland,Police Service of Northern Ireland,-5.678499,54.645899,On or near Cleland Park Central,NaN,NaN,Anti-social behaviour,NaN,NaN
2,NaN,2017-04,Police Service of Northern Ireland,Police Service of Northern Ireland,-5.876302,54.685471,On or near Shore Road,NaN,NaN,Anti-social behaviour,NaN,NaN
3,NaN,2017-04,Police Service of Northern Ireland,Police Service of Northern Ireland,-6.352118,54.854143,On or near Beechwood Drive,NaN,NaN,Anti-social behaviour,NaN,NaN
4,NaN,2017-04,Police Service of Northern Ireland,Police Service of Northern Ireland,-5.955758,54.674834,On or near,NaN,NaN,Anti-social behaviour,NaN,NaN
5,NaN,2017-04,Police Service of Northern Ireland,Police Service of Northern Ireland,-5.893519,54.206882,On or near Bryansford Road,NaN,NaN,Anti-social behaviour,NaN,NaN
6,NaN,2017-04,Police Service of Northern Ireland,Police Service of Northern Ireland,-7.468908,54.293414,On or near Harryville Street,NaN,NaN,Anti-social behaviour,NaN,NaN
7,NaN,2017-04,Police Service of Northern Ireland,Police Service of Northern Ireland,-6.215649,54.722103,On or near Renown Court,NaN,NaN,Anti-social behaviour,NaN,NaN
8,NaN,2017-04,Police Service of Northern Ireland,Police Service of Northern Ireland,-6.334844,54.464133,On or near Church Place,NaN,NaN,Anti-social behaviour,NaN,NaN
9,NaN,2017-04,Police Service of Northern Ireland,Police Service of Northern Ireland,-5.933119,54.580880,On or near Pretoria Street,NaN,NaN,Anti-social behaviour,NaN,NaN


In [50]:
df6['Reported by'].nunique()

1

In [51]:
df6['Falls within'].nunique()

1

In [60]:
df6['LSOA code'].count()

0

In [61]:
df6['LSOA name'].count()

0

In [57]:
df6['Crime type'].nunique()

14

In [63]:
df6['Last outcome category'].nunique()

0

In [88]:
src_dir ='C:/Users/jrbai/Jupyter/data/PSNI_StreetCrime_1Year'
frame = pd.DataFrame()
list_ = []
for root, dirs, files in os.walk(src_dir):
    allFiles = glob.glob(root + "/*.csv")
    for file_ in allFiles:
        df = pd.read_csv(file_,index_col=None, header=0)
        list_.append(df)
        frame = pd.concat(list_)
        print(len(frame))

14823
28731
44175
56345
69076
80913
92944
106737
119237
133168
146788
160782
175104
188409
202864
214271
226490
237948
249778
263103
276434
290752
304247
318513
332347
345701
359724
371860
384293
395878
406896
419604
432016
445745


In [89]:
frame

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context
0,NaN,2015-08,Police Service of Northern Ireland,Police Service of Northern Ireland,-7.378949,54.717334,On or near Dublin Street,NaN,NaN,Anti-social behaviour,NaN,NaN
1,NaN,2015-08,Police Service of Northern Ireland,Police Service of Northern Ireland,-5.891082,54.222501,On or near Bracken Avenue,NaN,NaN,Anti-social behaviour,NaN,NaN
2,NaN,2015-08,Police Service of Northern Ireland,Police Service of Northern Ireland,-5.667276,54.663573,On or near High Street,NaN,NaN,Anti-social behaviour,NaN,NaN
3,NaN,2015-08,Police Service of Northern Ireland,Police Service of Northern Ireland,-5.962330,54.587243,On or near Rodney Parade,NaN,NaN,Anti-social behaviour,NaN,NaN
4,NaN,2015-08,Police Service of Northern Ireland,Police Service of Northern Ireland,-5.894063,54.590423,On or near Brenda Street,NaN,NaN,Anti-social behaviour,NaN,NaN
5,NaN,2015-08,Police Service of Northern Ireland,Police Service of Northern Ireland,-5.952579,54.666532,On or near Glengormley Park,NaN,NaN,Anti-social behaviour,NaN,NaN
6,NaN,2015-08,Police Service of Northern Ireland,Police Service of Northern Ireland,-6.661786,54.348429,On or near,NaN,NaN,Anti-social behaviour,NaN,NaN
7,NaN,2015-08,Police Service of Northern Ireland,Police Service of Northern Ireland,-5.933398,54.597974,On or near College Street,NaN,NaN,Anti-social behaviour,NaN,NaN
8,NaN,2015-08,Police Service of Northern Ireland,Police Service of Northern Ireland,-5.930320,54.601597,On or near North Street,NaN,NaN,Anti-social behaviour,NaN,NaN
9,NaN,2015-08,Police Service of Northern Ireland,Police Service of Northern Ireland,-5.954752,54.589038,On or near,NaN,NaN,Anti-social behaviour,NaN,NaN


In [90]:
frame.count()

Crime ID                 274726
Month                    445745
Reported by              445745
Falls within             445745
Longitude                443370
Latitude                 443370
Location                 445745
LSOA code                     0
LSOA name                     0
Crime type               445745
Last outcome category         0
Context                       0
dtype: int64

In [ ]:
plt.bar(frame.Month, frame['Crime ID'].count())

KeyboardInterrupt: 